In [2]:
import random
import sys

In [3]:
def getNum(v) :

    n = len(v)
 
    index = random.randint(0, n - 1)
 
    num = v[index]
 
    v[index], v[n - 1] = v[n - 1], v[index]
    v.pop()
 
    return num

def GenerateRandomChromosome(NumberOfQueens):
    #[3,5,2,1,6,7,4,8]
    v = [0] * NumberOfQueens
    
    l = []
    for i in range(NumberOfQueens) :
        v[i] = i + 1

    while (len(v)) :
        l.append(getNum(v))

    return l


In [4]:
def fitness(chromosome):
    # horizontal_collisions = sum([chromosome.count(queen)-1 for queen in chromosome])/2
    diagonal_collisions = 0

    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter
    return int(maxFitness - diagonal_collisions) #28-(2+3)=23

In [18]:

def mostCollision(chromosome):
    # horizontal_collisions = sum([chromosome.count(queen)-1 for queen in chromosome])/2
    min = sys.maxsize
    max = -sys.maxsize - 1
    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        value = 0
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1
        value = left_diagonal[i + chromosome[i] - 1]
        value += right_diagonal[len(chromosome) - i + chromosome[i] - 2]
        if value > max:
            max = value
            maxIndex = i
        if value < min:
            min = value
            minIndex = i 

    return maxIndex

In [7]:
def parentSelection(chromosomeWithFitness):
    sortedPopulation = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'], reverse=True)
    return [sortedPopulation[0], sortedPopulation[1]]

In [6]:
def crossover(parents):
    
    mum = parents[0]['Chromosome']
    dad = parents[1]['Chromosome']

    size = len(mum)

    alice, bob = [-1] * size, [-1] * size
    start, end = sorted([random.randrange(size) for _ in range(2)])

    alice_inherited = []
    bob_inherited = []
    for i in range(start, end + 1):
        alice[i] = mum[i]
        bob[i] = dad[i]
        alice_inherited.append(mum[i])
        bob_inherited.append(dad[i])

    #print(alice, bob, alice_inherited, bob_inherited)

    current_dad_position, current_mum_position = 0, 0

    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        test_alice = alice[i]
        if test_alice==-1: #to be filled
            dad_trait = dad[current_dad_position]
            while dad_trait in alice_inherited:
                current_dad_position += 1
                dad_trait = dad[current_dad_position]
            alice[i] = dad_trait
            alice_inherited.append(dad_trait)

        i +=1
        
    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        test_bob = bob[i]
        if test_bob==-1:
            mum_trait = mum[current_mum_position]
            while mum_trait in bob_inherited:
                current_mum_position += 1
                mum_trait = mum[current_mum_position]
            bob[i] = mum_trait
            bob_inherited.append(mum_trait)

        i +=1

    return [alice, bob]
    

In [47]:
def mutate(childs):
    start1 = mostCollision(childs[0])
    start2 = mostCollision(childs[1])
    end1, end2 = [random.randrange(len(childs[0])) for _ in range(2)]
    childs[0][start1], childs[0][end1] = childs[0][end1], childs[0][start1]
    childs[1][start2], childs[1][end2] = childs[1][end2], childs[1][start2]

    return childs

In [48]:
if __name__ == "__main__":
    numberOfQueens = int(input("Enter Number of Queens: ")) #Input: Number of queens
    maxFitness = (numberOfQueens*(numberOfQueens-1))/2 # (8*(8-7))/2 = 28 
    population = [GenerateRandomChromosome(numberOfQueens) for _ in range(1000)]

    generation = 1
    chromosomeWithFitness = []
    for n in population:
        tempDict = {'Chromosome': n}
        tempDict['Fitness'] = fitness(n)
        chromosomeWithFitness.append(tempDict)
    print("Number Queens: ", numberOfQueens)
    print(max([chrom['Fitness'] for chrom in chromosomeWithFitness]))
    
while maxFitness not in [chrom['Fitness'] for chrom in chromosomeWithFitness]:
    parents = parentSelection(chromosomeWithFitness)
    # print(parents)
    childs = crossover(parents)
    
    # print("Before Mutataion: ",childs)
    # print(fitness(childs[0]), fitness(childs[1]))

    childs = mutate(childs)
    #print("After Mutataion: ",childs)
    #print(fitness(childs[0]), fitness(childs[1]))

    #replace the old population with new ones.
    chromosomeWithFitness = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'])
    chromosomeWithFitness[0], chromosomeWithFitness[1] = {'Chromosome': childs[0], 'Fitness': fitness(childs[0])}, {'Chromosome': childs[1], 'Fitness': fitness(childs[1])}

    generation += 1

finalList = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'], reverse=True)
print("Generations: ", generation, 'Chromosome: ', finalList[0]) #Output is number of generations and the N queen solution
    

Number Queens:  200
19819
Generations:  2083 Chromosome:  {'Chromosome': [46, 91, 130, 126, 131, 10, 90, 133, 58, 38, 112, 80, 152, 196, 182, 94, 12, 106, 129, 74, 53, 172, 33, 164, 101, 188, 9, 76, 184, 161, 71, 55, 75, 45, 121, 180, 28, 149, 114, 143, 108, 21, 191, 87, 37, 166, 179, 142, 109, 67, 169, 73, 110, 47, 17, 153, 148, 23, 3, 162, 35, 115, 59, 185, 50, 22, 54, 107, 44, 39, 136, 18, 160, 1, 89, 156, 66, 141, 95, 99, 24, 116, 199, 183, 178, 49, 157, 72, 163, 128, 36, 123, 5, 145, 174, 11, 98, 190, 194, 125, 31, 138, 42, 82, 14, 100, 27, 120, 19, 34, 193, 26, 146, 173, 170, 4, 119, 41, 177, 56, 187, 40, 81, 150, 197, 13, 25, 134, 151, 78, 20, 167, 88, 198, 159, 65, 30, 105, 189, 77, 154, 96, 102, 79, 2, 57, 118, 48, 135, 127, 104, 181, 15, 137, 16, 29, 97, 8, 111, 132, 158, 165, 6, 86, 155, 139, 93, 43, 68, 168, 186, 63, 200, 192, 83, 140, 176, 52, 85, 62, 122, 7, 171, 64, 61, 124, 195, 84, 117, 147, 70, 32, 60, 175, 144, 51, 92, 103, 69, 113], 'Fitness': 19900}
